In [1]:
import easyocr
import cv2
import numpy as np
import csv
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, f1_score

In [2]:
# Fungsi untuk membaca dan memproses gambar, lalu mengenali teks
def recognize_text(image_path):
    reader = easyocr.Reader(['en'], gpu=True)  # Memuat sekali saja ke memori.
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    sharpen_kernel = np.array([[-1, -1, -1], [-1, 9, -1], [-1, -1, -1]])
    sharpen = cv2.filter2D(gray, -1, sharpen_kernel)
    thresh = cv2.threshold(sharpen, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
    kernel = np.ones((3, 3), np.uint8)
    morph = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
    r_easy_ocr = reader.readtext(morph, detail=0)
    recognized_text = " ".join(r_easy_ocr)
    return recognized_text

In [3]:

# Fungsi untuk membaca ground truth dari file CSV
def read_ground_truth(csv_file):
    ground_truth_dict = {}
    with open(csv_file, mode='r') as file:
        csv_reader = csv.reader(file)
        next(csv_reader)  # Lewati header CSV
        for row in csv_reader:
            image_name = row[0]
            ground_truth = row[1]
            ground_truth_dict[image_name] = ground_truth
    return ground_truth_dict

In [4]:
def read_ground_truth(csv_file):
    ground_truth_dict = {}
    with open(csv_file, mode='r') as file:
        csv_reader = csv.reader(file)
        next(csv_reader)  # Lewati header CSV
        for row in csv_reader:
            image_name = row[0]
            ground_truth = row[1]
            ground_truth_dict[image_name] = ground_truth
    return ground_truth_dict

In [5]:
# Fungsi untuk menghitung akurasi per karakter
def calculate_char_accuracy(recognized_text, ground_truth):
    recognized_text = recognized_text.upper()
    ground_truth = ground_truth.upper()
    TP = sum(1 for i in range(min(len(recognized_text), len(ground_truth))) if recognized_text[i] == ground_truth[i])
    FP = sum(1 for i in range(len(recognized_text)) if i >= len(ground_truth) or recognized_text[i] != ground_truth[i])
    FN = sum(1 for i in range(len(ground_truth)) if i >= len(recognized_text) or recognized_text[i] != ground_truth[i])
    precision = TP / (TP + FP) if (TP + FP) != 0 else 0
    recall = TP / (TP + FN) if (TP + FN) != 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0
    accuracy = TP / max(len(recognized_text), len(ground_truth)) * 100
    return accuracy, f1, TP, FP, FN

In [6]:
# Fungsi untuk memplot confusion matrix
def plot_confusion_matrix(y_true, y_pred, labels):
    cm = confusion_matrix(y_true, y_pred, labels=labels)
    plt.figure(figsize=(10, 7))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title('Confusion Matrix easyOCR')
    plt.colorbar()
    tick_marks = np.arange(len(labels))
    plt.xticks(tick_marks, labels, rotation=45)
    plt.yticks(tick_marks, labels)
    plt.ylabel('True Labels')
    plt.xlabel('Predicted Labels')
    for i in range(len(labels)):
        for j in range(len(labels)):
            plt.text(j, i, cm[i, j], ha='center', va='center', color='red')
    plt.tight_layout()
    plt.show()

In [7]:
# Fungsi untuk memplot grafik akurasi
def plot_accuracy_graph(accuracies, ground_truth_lengths):
    plt.figure(figsize=(10, 6))
    plt.plot(ground_truth_lengths, accuracies, marker='o', linestyle='-', color='b', label='Akurasi per Gambar')
    plt.xlabel('Panjang String Ground Truth')
    plt.ylabel('Akurasi (%)')
    plt.title('Akurasi easyOCR')
    plt.grid(True)
    plt.legend()
    plt.show()

In [8]:

# Fungsi untuk memplot grafik F1 Score
def plot_f1_graph(f1_scores):
    plt.figure(figsize=(10, 6))
    plt.plot(range(1, len(f1_scores) + 1), f1_scores, marker='o', linestyle='-', color='r', label='F1 Score per Gambar')
    plt.xlabel('Nomor Gambar')
    plt.ylabel('F1 Score')
    plt.title('F1 Convidence easyOCR')
    plt.grid(True)
    plt.legend()
    plt.show()

In [9]:
# Fungsi untuk memplot grafik akurasi terhadap nomor iterasi dengan rata-rata
def plot_accuracy_vs_iteration(accuracies):
    # Menghitung rata-rata kumulatif
    avg_accuracies = [sum(accuracies[:i + 1]) / (i + 1) for i in range(len(accuracies))]
    
    plt.figure(figsize=(10, 6))
    plt.plot(range(1, len(accuracies) + 1), accuracies, marker='o', linestyle='-', color='g', label='Akurasi per Iterasi')
    plt.plot(range(1, len(avg_accuracies) + 1), avg_accuracies, marker='', linestyle='--', color='b', label='Rata-rata Kumulatif')
    plt.xlabel('Nomor Iterasi')
    plt.ylabel('Akurasi (%)')
    plt.title('Akurasi easyOCR 30 Char dengan Rata-rata Kumulatif') # ganti dengan kebutuhan
    plt.grid(True)
    plt.legend()
    plt.show()


In [10]:

# Membaca ground truth dari file CSV
ground_truth_file = '../Capital/26-30/GROUND_TRUTH30.csv' # ganti dengan path file ground truth yang diinginkan 
ground_truth_dict = read_ground_truth(ground_truth_file)

all_true = []
all_pred = []
accuracies = []
f1_scores = []  # Menyimpan F1 Score per gambar
ground_truth_lengths = []

In [ ]:
for i in range(1, 11):
    image_path = f'../Capital/26-30/TEST_{i}.png' # ganti dengan path file gambar yang diinginkan
    recognized_text = recognize_text(image_path)
    image_name = f'TEST_{i}.png'
    ground_truth = ground_truth_dict.get(image_name, "")
    accuracy, f1, TP, FP, FN = calculate_char_accuracy(recognized_text, ground_truth)
    all_true.extend(list(ground_truth.upper()))
    if recognized_text:
        all_pred.extend(list(recognized_text.upper()))
    else:
        all_pred.extend([''] * len(ground_truth))
    accuracies.append(accuracy)
    f1_scores.append(f1)  # Menyimpan F1 score untuk gambar ini
    ground_truth_lengths.append(len(ground_truth))
    print(f"Gambar {i} - Akurasi: {accuracy:.2f}% - F1 Score: {f1:.2f} - TP: {TP} - FP: {FP} - FN: {FN}")

if len(all_true) > len(all_pred):
    all_pred.extend([''] * (len(all_true) - len(all_pred)))
elif len(all_true) < len(all_pred):
    all_true.extend([''] * (len(all_pred) - len(all_true)))
 # Menampilkan grafik F1 Score per gambar


In [ ]:
conf_matrix = confusion_matrix(all_true, all_pred, labels=list(set(all_true + all_pred)))
f1_total = f1_score(all_true, all_pred, average='weighted')

print("\nConfusion Matrix:")
print(conf_matrix)
print(f"Total F1 Score (Weighted): {f1_total:.2f}")

labels = sorted(set(all_pred))
plot_confusion_matrix(all_true, all_pred, labels)
# plot_accuracy_graph(accuracies, ground_truth_lengths)
plot_f1_graph(f1_scores)  # Menampilkan grafik F1 Score per gambar
plot_accuracy_vs_iteration(accuracies)  # Menampilkan grafik akurasi terhadap nomor iterasi